# Homework 1
By Quentin Barnes  
CS 344

### Part one: Introspection:
Introspection is the practice of looking inward, towards one’s own thoughts in an attempt to gain insight on how we work.  It is a practice that is as old as Plato and is very useful in the field of psychology.  Because of this, it is useful in modeling for AI, however only to an extent.  It is useful because a lot of the times we can teach the computers to solve the problems the same ways that we do.  This does have some limitations and drawbacks, though, since computers are mechanical in nature.  Sometimes the way that we do things in our heads is very inefficient because we do have limitations.  
  
I think that an interesting area to look at is vision.  As we are teaching computers to actually see, we are doing it in a way that is similar to how we learned to see.  For computers we have people label a bunch of objects and then feed it to programs to let them learn what those objects look like, which is a lot like how humans learn to recognize things.  However, even in this case, they don’t do it exactly the same, and the program can also be fooled a lot easier.  

### Part 2: Traveling Salesman Probelm: 

In [4]:
"""
This module implements local search on the Traveling Salesman Problem.  It uses
simulated annealing and hill climbing to do this.  It will randomly create a city
that has a list of linked cities in it.  It will then use the above algorithms to
try and find the shortest path through them.

@author: Quentin Barnes
"""
import math
import time
from search import Problem, hill_climbing, simulated_annealing, \
    exp_schedule, genetic_search
from random import randrange, shuffle


class TPSVariant(Problem):
    """
    State: Current order of citys
    Action: Take the city with the longest distance to the next one, then make a list
    of all the other cities swaped with that one.
    Value: Total distance took to travel between the cities
    """

    # Sets the initial state, the size of the city, the configuration of the cities, and the maximum distance
    def __init__(self, initial, citySize, cityConfig, maximum, connectHome):
        self.initial = initial[:]
        self.citySize = citySize
        self.cityConfig = cityConfig
        self.maximum = maximum
        self.connectHome = connectHome

    # It takes the city with the max distance to the next one and then
    # makes a list of that city swaped with each other city as the actions
    def actions(self, state):
        max = self.getMaxDistNode(state)
        options = []
        for i in range(self.citySize):
            if i != max:
                newOption = state[:]
                newOption[i], newOption[max] = newOption[max], newOption[i]
                options.append(newOption[:])
        return options[:]

    # Result is the action taken
    def result(self, state, action):
        return action[:]

    # Inverts the realValue since the algorithms are looking for a maximum,
    # while we are looking for the minimum distance
    def value(self, state):
        return self.maximum - self.realValue(state)

    # Uses the city configuration to sum up the total distance that the salesman travels
    def realValue(self, state):
        totalDist = 0
        for i in range(self.citySize - 1):
            totalDist += self.cityConfig[state[i]][state[i + 1]]
        if self.connectHome:
            totalDist += self.cityConfig[self.citySize-1][1]
        return totalDist

    # Finds and returns the index of the node with the max distance
    def getMaxDistNode(self, state):
        indexOfMax = 0
        lengthOfMax = 0
        for n in range(self.citySize - 1):
            if self.cityConfig[state[n]][state[n + 1]] > lengthOfMax:
                indexOfMax = n
                lengthOfMax = self.cityConfig[state[n]][state[n + 1]]
        return indexOfMax


if __name__ == '__main__':

    # Sets the values of how large the city is and the max distance between them
    # Also sets if the problem should be a loop or not
    fillInSize = citySize = 20
    maxDistance = 20
    connectHome = False

    # Creates the city configuration
    maximum = citySize * maxDistance
    cityConfig = [[0 for x in range(citySize)] for y in range(citySize)]
    for i in range(citySize):
        fillInSize -= 1
        for j in range(fillInSize):
            cityConfig[i + j + 1][i] = cityConfig[i][i +
                                                     j + 1] = randrange(1, maxDistance, 1)

    # Creates an initial state
    initial = [x for x in range(citySize)]
    shuffle(initial)

    p = TPSVariant(initial, citySize, cityConfig, maximum, connectHome)

    time1 = time.time()
    hill_solution = hill_climbing(p)
    time2 = time.time()

    simAneal = simulated_annealing(
        p,
        exp_schedule(k=20, lam=0.005, limit=1000)
    )
    time3 = time.time()

    print("Initial: \t\t\t\t\t", initial, "  Distance: ", p.realValue(initial))
    print("Hill Climbing solution: \t", hill_solution,
          "  Distance: ", p.realValue(hill_solution), "  Time: ", (time2 - time1))
    print("Sim Anneal solution: \t\t", simAneal,
          "  Distance: ", p.realValue(simAneal), "  Time: ", (time3 - time2))


Initial: 		 [3, 8, 9, 1, 11, 12, 16, 4, 10, 2, 6, 17, 18, 13, 15, 7, 14, 5, 19, 0]   Distance:  196
Hill Climbing solution:  [2, 17, 9, 18, 11, 16, 15, 4, 3, 1, 19, 6, 13, 12, 10, 7, 14, 5, 8, 0]   Distance:  59   Time:  0.0029489994049072266
Sim Anneal solution: 	 [14, 5, 6, 16, 10, 9, 17, 3, 2, 13, 8, 0, 11, 12, 4, 7, 18, 15, 19, 1]   Distance:  60   Time:  0.0835409164428711


This program is creates a random map and then uses hill climbing and simulated annealing to try and solve the problem.  The states for the problem are lists of the order that the cities will be traveld to.  It was unclear whether the salesman was supposed to travel back to where he started in the end or not, so I made it a option for the program.  The actions that I chose were 